___

<a href='http://www.pieriandata.com'><img src='../Pierian_Data_Logo.png'/></a>
___
<center><em>Авторские права принадлежат Pierian Data Inc.</em></center>
<center><em>Для дополнительной информации посетите наш сайт <a href='http://www.pieriandata.com'>www.pieriandata.com</a></em></center>

# Извлечение признаков из текста

Этот блокнот разделён на две части:
* Сначала мы выясним, что нам понадобится для построения инструментария NLP (Natural Language Processing), который превратит набор текста в числовой массив из *признаков*. Для этого мы вручную вычислим, как часто встречаются те или иные слова, и построим метрику TF-IDF.
* Далее мы выполним эти шаги с помощью scikit-learn.

# Часть 1: основные принципы извлечения признаков из текста


В этой части мы с помощью базовых операций Python построим очень простую систему NLP. Мы возьмём *набор документов* (*corpus of documents*) - это будет два текстовых файла. Далее создадим *словарь* (*vocabulary*) из всех слов обоих документов. И затем посмотрим на технику *Мешок слов* (*Bag of Words*) для извлечения признаков из каждого документа.<br>
<div class="alert alert-info" style="margin: 20px">В этом разделе мы только приводим иллюстрации основных принципов!
<br>Не обращайте внимание на то, как пишется код в Python - позднее мы применим для этой задачи Scikit-Learn.</div>

## Начнём с документов:
Для простоты изложения в наших текстовых файлах One.txt и Two.txt не будет каких-либо знаков пунктуации. Давайте откроем эти файлы и прочитаем данные. Обратите внимание, что если в будущем у Вас файлы будут большие, то их не следует выводить на экран полностью.


In [57]:
with open('One.txt') as mytext:
    print(mytext.read())

This is a story about dogs
our canine pets
Dogs are furry animals



In [58]:
with open('Two.txt') as mytext:
    print(mytext.read())

This story is about surfing
Catching waves is fun
Surfing is a popular water sport



### Читаем весь текст как строку string

In [59]:
with open('One.txt') as mytext:
    entire_text = mytext.read()

In [60]:
entire_text

'This is a story about dogs\nour canine pets\nDogs are furry animals\n'

In [61]:
print(entire_text)

This is a story about dogs
our canine pets
Dogs are furry animals



### Читаем каждую строку файла отдельно и помещаем в список

In [62]:
with open('One.txt') as mytext:
    lines = mytext.readlines()

In [63]:
lines

['This is a story about dogs\n',
 'our canine pets\n',
 'Dogs are furry animals\n']

### Читаем отдельные слова

In [64]:
with open('One.txt') as f:
    words = f.read().lower().split()

In [65]:
words

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

## Создаём словарь vocabulary (Мешок слов - "Bag of Words")

Для этого мы возьмём все слова из обоих документов, найдём уникальные слова, и пронумеруем их.

In [83]:
with open('One.txt') as f:
    words_one = f.read().lower().split()

In [84]:
words_one

['this',
 'is',
 'a',
 'story',
 'about',
 'dogs',
 'our',
 'canine',
 'pets',
 'dogs',
 'are',
 'furry',
 'animals']

In [85]:
len(words_one)

13

In [86]:
uni_words_one = set(words)

In [87]:
uni_words_one

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'dogs',
 'furry',
 'is',
 'our',
 'pets',
 'story',
 'this'}

**То же самое для файла Two.txt**

In [88]:
with open('Two.txt') as f:
    words_two = f.read().lower().split()
    uni_words_two = set(words_two)

In [89]:
uni_words_two

{'a',
 'about',
 'catching',
 'fun',
 'is',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

**Получаем все уникальные слова из всех документов**

In [91]:
all_uni_words = set()
all_uni_words.update(uni_words_one)
all_uni_words.update(uni_words_two)

In [93]:
all_uni_words

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'catching',
 'dogs',
 'fun',
 'furry',
 'is',
 'our',
 'pets',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

In [94]:
full_vocab = dict()
i = 0

for word in all_uni_words:
    full_vocab[word] = i
    i = i+1

In [96]:
# Эти слова могут быть НЕ в алфавитном порядке! 
# Цикл for вовсе НЕ обязан проходит по множеству set() в алфавитном порядке!
full_vocab

{'water': 0,
 'sport': 1,
 'canine': 2,
 'pets': 3,
 'about': 4,
 'catching': 5,
 'dogs': 6,
 'furry': 7,
 'fun': 8,
 'story': 9,
 'is': 10,
 'our': 11,
 'surfing': 12,
 'animals': 13,
 'are': 14,
 'this': 15,
 'popular': 16,
 'a': 17,
 'waves': 18}

## Мешок слов - считаем, как часто встречаются отдельные слова

После того, как мы создали словарь из слов, давайте выполним *извлечение признаков* для каждого из двух исходных документов:

**Создаём пустые счётчики для каждого документа**

In [126]:
# Для каждого документа создаём пустые счётчики для всех слов из словаря full_vocab:
one_freq = [0]*len(full_vocab)
two_freq = [0]*len(full_vocab)
all_words = ['']*len(full_vocab)

In [127]:
one_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [128]:
two_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [129]:
all_words

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [130]:
for word in full_vocab:
    word_ind = full_vocab[word]
    all_words[word_ind] = word    

In [131]:
all_words

['water',
 'sport',
 'canine',
 'pets',
 'about',
 'catching',
 'dogs',
 'furry',
 'fun',
 'story',
 'is',
 'our',
 'surfing',
 'animals',
 'are',
 'this',
 'popular',
 'a',
 'waves']

**Добавляем счётчики слов для каждого документа:**

In [132]:
# Берём слова из файла, для каждого слова ищем соответствие в нашем словаре full_vocab:
with open('One.txt') as f:
    one_text = f.read().lower().split()
    
for word in one_text:
    word_ind = full_vocab[word]
    one_freq[word_ind]+=1

In [133]:
one_freq

[0, 0, 1, 1, 1, 0, 2, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0]

In [134]:
# То же самое для второго документа:
with open('Two.txt') as f:
    two_text = f.read().lower().split()
    
for word in two_text:
    word_ind = full_vocab[word]
    two_freq[word_ind]+=1

In [135]:
two_freq

[1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 3, 0, 2, 0, 0, 1, 1, 1, 1]

In [141]:
pd.DataFrame(data=[one_freq,two_freq],columns=all_words)

,water,sport,canine,pets,about,catching,dogs,furry,fun,story,is,our,surfing,animals,are,this,popular,a,waves
0,0,0,1,1,1,0,2,1,0,1,1,1,0,1,1,1,0,1,0
1,1,1,0,0,1,1,0,0,1,1,3,0,2,0,0,1,1,1,1


Некоторые слова есть в обоих документах, а некоторые слова есть только в `One.txt`, и некоторые другие слова есть только в `Two.txt`. Если мы применим эту логику для десятков тысяч документов, то наш словарь вполне сможет вырасти до десятков тысяч слов. При этом матрицы будут содержать очень много нулей - это будут **разреженные метрицы** (**sparse matrices**).


# Продолжение:

## Мешок слов и Tf-idf
В приведённом выше примере, каждый вектор можно рассматривать как мешок слов (*bag of words*). Сами по себе эти вектора не очень полезны, но мы также можем добавить к ним частоту слов (*term frequencies - TF*) - как часто то или иное слово встречается в документе. Простой способ сделать это - это посчитать, сколько раз встречается слово в документе, и разделить на общее количество слов в документе. Тогда можно сравнивать между большими и маленькими документами, сколько раз то или иное слово встречается в документе.

Однако, если какое-то слово встречается во многих документах, то такое слово не будет являться хорошим признаком для отделения документов друг от друга. Чтобы работать с такими словами, можно добавить метрику *inverse document frequency (IDF)*, которая вычисляется как общее количество документов, разделить на количество документов, в которых содержится рассматриваемое нами слово (слово, для которого вычисляется IDF). В практических задачах это значение приводится к логарифмической шкале, подробнее см. [здесь](https://ru.wikipedia.org/wiki/TF-IDF).

Соединяя метрики TF (Term Frequency) и IDF (Inverse Document Frequency), мы получаем метрику [**tf-idf**](https://ru.wikipedia.org/wiki/TF-IDF).

## Стоп-слова и морфология слов
Некоторые слова встречаются слишком часто, например слова "the" и "and" в английском языке. Эти слова можно просто исключить из рассмотрения. 

Кроме этого, одно и то же слово может встречаться в единственном и множественном числе, а также в различных падежах. Нам бы хотелось записать это слово один раз, а не записывать различные его вариации (например, записать только `cat` для обоих слов `cat` и `cats`). Это позволит нам уменьшить размер словаря и увеличить скорость работы модели.

С другой стороны, различные падежи слов - особенно в русском языке - несут дополнительную смысловую информацию. Эту информацию по-хорошему можно сохранять, то есть ссылаться на базовое слово в словаре, плюс сохранять дополнительные атрибуты этого слова в конкретном месте текста. В нашем блокноте мы ограничимся лишь базовым подсчётом частоты слов - как в отдельном документе, так и в целом наборе документов.

# Часть 2: извлечение признаков и текста в Scikit-Learn

Давайте применим sklearn для решения этих задач!

# Варианты извлечения признаков в Scikit-Learn

In [185]:
text = ['This is a line',
           "This is another line",
       "Completely different line"]

## CountVectorizer

In [186]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer

In [187]:
cv = CountVectorizer()

In [188]:
cv.fit_transform(text)

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [189]:
sparse_mat = cv.fit_transform(text)

In [190]:
sparse_mat.todense()

matrix([[0, 0, 0, 1, 1, 1],
        [1, 0, 0, 1, 1, 1],
        [0, 1, 1, 0, 1, 0]], dtype=int64)

In [191]:
cv.vocabulary_

{'this': 5, 'is': 3, 'line': 4, 'another': 0, 'completely': 1, 'different': 2}

In [192]:
cv = CountVectorizer(stop_words='english')

In [193]:
cv.fit_transform(text).todense()

matrix([[0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]], dtype=int64)

In [194]:
cv.vocabulary_

{'line': 2, 'completely': 0, 'different': 1}

## TfidfTransformer

TfidfVectorizer применяется для текстовых документов, а TfidfTransformer применяется к матрице со счётчиками, которую возвращает CountVectorizer

In [206]:
tfidf_transformer = TfidfTransformer()

In [207]:
cv = CountVectorizer()

In [208]:
counts = cv.fit_transform(text)

In [209]:
counts

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [210]:
tfidf = tfidf_transformer.fit_transform(counts)

In [211]:
tfidf.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

In [212]:
from sklearn.pipeline import Pipeline

In [215]:
pipe = Pipeline([('cv',CountVectorizer()),('tfidf',TfidfTransformer())])

In [219]:
results = pipe.fit_transform(text)

In [220]:
results

<3x6 sparse matrix of type '<class 'numpy.float64'>'
	with 10 stored elements in Compressed Sparse Row format>

In [218]:
results.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

## TfIdfVectorizer

Выполняем оба действия единым шагом!

In [202]:
tfidf = TfidfVectorizer()

In [203]:
new = tfidf.fit_transform(text)

In [204]:
new.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])

In [17]:
with open('One.txt') as mytext:
    words_one = mytext.read().lower().split()
    uni_words_one = set(words_one)

In [21]:
uni_words_one

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'dogs',
 'furry',
 'is',
 'our',
 'pets',
 'story',
 'this'}

In [24]:
with open('Two.txt') as mytext:
    words_two = mytext.read().lower().split()
    uni_words_two = set(words_two)

In [25]:
uni_words_two

{'a',
 'about',
 'catching',
 'fun',
 'is',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

In [26]:
all_uni_words = set()

In [27]:
all_uni_words.update(uni_words_one)
all_uni_words.update(uni_words_two)

In [28]:
all_uni_words

{'a',
 'about',
 'animals',
 'are',
 'canine',
 'catching',
 'dogs',
 'fun',
 'furry',
 'is',
 'our',
 'pets',
 'popular',
 'sport',
 'story',
 'surfing',
 'this',
 'water',
 'waves'}

In [35]:
full_vocab = dict()
i = 0 

for n, word in enumerate(all_uni_words):
    full_vocab[word] = n

In [36]:
full_vocab

{'waves': 0,
 'are': 1,
 'fun': 2,
 'this': 3,
 'pets': 4,
 'furry': 5,
 'water': 6,
 'surfing': 7,
 'our': 8,
 'story': 9,
 'dogs': 10,
 'is': 11,
 'a': 12,
 'about': 13,
 'canine': 14,
 'catching': 15,
 'animals': 16,
 'popular': 17,
 'sport': 18}

In [37]:
one_freq = [0] * len(full_vocab)

In [42]:
two_freq = [0] * len(full_vocab)

In [43]:
all_words = [''] * len(full_vocab)

In [44]:
one_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [45]:
two_freq

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [46]:
all_words

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [47]:
with open('One.txt') as f:
    one_text = f.read().lower().split()

In [63]:
for word in one_text:
    word_ind = full_vocab[word]
    one_freq[word_ind] += 1

In [64]:
one_freq

[0, 8, 0, 8, 8, 8, 0, 0, 8, 8, 16, 8, 8, 8, 8, 0, 8, 0, 0]

In [67]:
with open('One.txt') as f:
    two_text = f.read().lower().split()

In [68]:
for word in two_text:
    word_ind = full_vocab[word]
    two_freq[word_ind] += 1

In [69]:
two_freq

[0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0, 0]

In [70]:
all_words

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

In [71]:
for word in full_vocab:
    word_ind = full_vocab[word]
    all_words[word_ind] = word

In [72]:
all_words

['waves',
 'are',
 'fun',
 'this',
 'pets',
 'furry',
 'water',
 'surfing',
 'our',
 'story',
 'dogs',
 'is',
 'a',
 'about',
 'canine',
 'catching',
 'animals',
 'popular',
 'sport']

In [74]:
import pandas as pd

In [76]:
bow = pd.DataFrame(data=[one_freq, two_freq], columns = all_words)

In [84]:
bow = bow.replace({16: 2, 8: 1})

In [85]:
bow

,waves,are,fun,this,pets,furry,water,surfing,our,story,dogs,is,a,about,canine,catching,animals,popular,sport
0,0,1,0,1,1,1,0,0,1,1,2,1,1,1,1,0,1,0,0
1,0,1,0,1,1,1,0,0,1,1,2,1,1,1,1,0,1,0,0


In [86]:
text = ['This is a line',
        'This is another line',
        'Completely different line']

In [113]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

In [114]:
help(CountVectorizer)

Help on class CountVectorizer in module sklearn.feature_extraction.text:

class CountVectorizer(_VectorizerMixin, sklearn.base.BaseEstimator)
 |  CountVectorizer(*, input='content', encoding='utf-8', decode_error='strict', strip_accents=None, lowercase=True, preprocessor=None, tokenizer=None, stop_words=None, token_pattern='(?u)\\b\\w\\w+\\b', ngram_range=(1, 1), analyzer='word', max_df=1.0, min_df=1, max_features=None, vocabulary=None, binary=False, dtype=<class 'numpy.int64'>)
 |
 |  Convert a collection of text documents to a matrix of token counts.
 |
 |  This implementation produces a sparse representation of the counts using
 |  scipy.sparse.csr_matrix.
 |
 |  If you do not provide an a-priori dictionary and you do not use an analyzer
 |  that does some kind of feature selection then the number of features will
 |  be equal to the vocabulary size found by analyzing the data.
 |
 |  For an efficiency comparison of the different feature extractors, see
 |  :ref:`sphx_glr_auto_examp

In [96]:
cv = CountVectorizer(stop_words='english')

In [105]:
sparse_matrix = cv.fit_transform(text)

In [106]:
sparse_matrix.todense()

matrix([[0, 0, 1],
        [0, 0, 1],
        [1, 1, 1]], dtype=int64)

In [99]:
cv.vocabulary_

{'line': 2, 'completely': 0, 'different': 1}

In [100]:
cv.vocabulary_

{'line': 2, 'completely': 0, 'different': 1}

In [103]:
tfidf = TfidfTransformer()

In [107]:
sparse_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 5 stored elements and shape (3, 3)>

In [109]:
results = tfidf.fit_transform(sparse_matrix)

In [111]:
results.todense()

matrix([[0.        , 0.        , 1.        ],
        [0.        , 0.        , 1.        ],
        [0.65249088, 0.65249088, 0.38537163]])

In [116]:
tv = TfidfVectorizer()

In [117]:
tv_results = tv.fit_transform(text)

In [118]:
tv_results

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10 stored elements and shape (3, 6)>

In [119]:
tv_results.todense()

matrix([[0.        , 0.        , 0.        , 0.61980538, 0.48133417,
         0.61980538],
        [0.63174505, 0.        , 0.        , 0.4804584 , 0.37311881,
         0.4804584 ],
        [0.        , 0.65249088, 0.65249088, 0.        , 0.38537163,
         0.        ]])